# Data pre-processing

Para tener los datos necesario para probar los enfoques de filtros colaborativos(CF) y basados en contenido(CB) necesito:

* Calificaciones de los ítems(movies) de los usuarios (CF)
* Features propies de los ítems (CB)

Dado esto, a continuacion se combinaran los siguientes datasets:

* MovieLens 25M Dataset: Prácticamente no tiene información de las películas pero si tiene las calificaciones de los usuarios.
* TMDB Movie Dataset: No tiene calificaciones personalizadas como el dataset anterior pero tiene varios features para las películas que es lo que necesito.

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

BASE_PATH       = '..'
DATASETS_PATH   = f'{BASE_PATH}/datasets'
MOVIE_LENS_PATH = f'{DATASETS_PATH}/ml-25m'
TMDB_PATH       = f'{DATASETS_PATH}/tmdb'

In [2]:
import sys
sys.path.append(f'{BASE_PATH}/lib')

from database.mongo import Mongo
from pytorch_common.util import LoggerBuilder

In [3]:
LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

# Pre-processing steps

#### 1. Import five collecitons to **movies** mongodb database:

    * From movie lens dataset:
        * rattings
        * movies
        * links
        * tags
    * From the movie database dataset:
        * movies_metadata

In [ ]:
!mkdir -p {DATASETS_PATH}

!cd {DATASETS_PATH}; curl -LO http://files.grouplens.org/datasets/movielens/ml-25m.zip

!cd {DATASETS_PATH}; unzip -o ml-25m.zip

In [ ]:
!mkdir -p {TMDB_PATH}

Download TMDB dataset from [Here](https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata) (archive.zip file) into next directory:

In [ ]:
TMDB_PATH

In [ ]:
!cd {TMDB_PATH}; unzip -o archive.zip
!cd {TMDB_PATH}; mv tmdb_5000_movies.csv movies_metadata.csv

In [4]:
MOVIE_LENS_FILES = [ f'{MOVIE_LENS_PATH}/{f}' for f in [ 
    'ratings.csv',
    'movies.csv',
    'links.csv',
    'tags.csv'
]]
TMDB_FILES = [ f'{TMDB_PATH}/movies_metadata.csv']

In [5]:
MOVIE_LENS_FILES

['../datasets/ml-25m/ratings.csv',
 '../datasets/ml-25m/movies.csv',
 '../datasets/ml-25m/links.csv',
 '../datasets/ml-25m/tags.csv']

In [6]:
TMDB_FILES

['../datasets/tmdb/movies_metadata.csv']

In [25]:
Mongo.import_csv(database = 'movies', file_paths = ['../datasets/ml-25m/links.csv'])

2023-11-05 14:25:48,068 - INFO - Success: b'2023-11-05T14:25:46.886-0300  connected to: mongodb://localhost/2023-11-05T14:25:48.062-0300  62423 document(s) imported successfully. 0 document(s) failed to import.'


In [7]:
Mongo.import_csv(database = 'movies', file_paths = TMDB_FILES)

2023-11-05 14:00:50,844 - INFO - Success: b'2023-11-05T14:00:50.590-0300  connected to: mongodb://localhost/2023-11-05T14:00:50.843-0300  4803 document(s) imported successfully. 0 document(s) failed to import.'


In [31]:
Mongo.import_csv(database = 'movies', file_paths = MOVIE_LENS_FILES)

2023-11-05 12:02:59,602 - INFO - import ratings collection...
2023-11-05 12:02:59,607 - INFO - import movies collection...
2023-11-05 12:02:59,613 - INFO - import links collection...
2023-11-05 12:02:59,619 - INFO - import tags collection...
2023-11-05 12:10:10,289 - INFO - b'2023-11-05T12:02:59.623-0300  connected to: mongodb://localhost/2023-11-05T12:03:02.624-0300  [........................] movies.ratings  8.36MB/647MB (1.3%)2023-11-05T12:03:05.624-0300  [........................] movies.ratings  16.9MB/647MB (2.6%)2023-11-05T12:03:08.624-0300  [........................] movies.ratings  25.1MB/647MB (3.9%)2023-11-05T12:03:11.624-0300  [#.......................] movies.ratings  30.4MB/647MB (4.7%)2023-11-05T12:03:14.624-0300  [#.......................] movies.ratings  34.7MB/647MB (5.4%)2023-11-05T12:03:17.624-0300  [#.......................] movies.ratings  39.1MB/647MB (6.0%)2023-11-05T12:03:20.625-0300  [#.......................] movies.ratings  43.2MB/647MB (6.7%)2023-11-05T12:0

#### 2. Transform imdb id to number

In [27]:
Mongo.command('movies',  """
db.getCollection('movies_metadata').aggregate([
    {
        $match: { 
            $and: [
                { imdb_id: { $ne: "" } },
                { imdb_id: { $ne: 0 } }
            ]
        }
    },
    {
        $addFields: {
            imdb_id: {$toLong: [ { $arrayElemAt: [ { $split: ["$imdb_id", "tt"]}, 1 ] }] }
        }
    },
    { $out: "movies_metadata_v2" }
]);
""")

2023-11-05 14:26:34,241 - INFO - Success: b'(node:75135) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  6547d0499c3586292fb00427Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:75135) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 3.  Add indexes to both links and movies_metadata_v2 collections.

In [26]:
Mongo.command('movies', """
db.getCollection('links').createIndex(
    { "movieId": 1 }, 
    { 
        unique: true, 
        name: "movieId_unique_index"
    }
);
""")

2023-11-05 14:26:01,291 - INFO - Success: b'(node:75021) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  6547d028b3d368b37ca4522fConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:75021) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [60]:
Mongo.command('movies', """
// Group doc ids with same user_movie_id.
duplicates = db.getCollection('movies_metadata_v2').aggregate([
  {
    $group: {
      _id:   { field: "$user_movie_id" },
      count: { $sum: 1 },
      docs:  { $push: "$_id" }
    }
  },
  {
    $match: {
      count: { $gt: 1 }
    }
  }
]);


// Paso 2: Crea una instancia de Bulk y agrega las operaciones de eliminación
var bulk = db.getCollection('movies_metadata_v2').initializeUnorderedBulkOp();

duplicates.forEach(function(duplicate) {
  var idsToDelete = duplicate.docs.slice(1); // conserva el primer documento y elimina los demás
  bulk.find({ _id: { $in: idsToDelete } }).remove();
});

// Paso 3: Ejecuta las operaciones de eliminación en lote
bulk.execute();
""")

2023-11-05 21:31:19,994 - INFO - Success: b'(node:58489) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654833d710560de987cbba4cConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:58489) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [61]:
Mongo.command('movies', """
db.getCollection('movies_metadata_v2').createIndex(
    { 
        "imdb_id": 1
    }, 
    {
        unique: true,
        name: "imdb_id_multiple_index"
    }
);
""")

2023-11-05 21:31:25,231 - INFO - Success: b'(node:58507) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654833dcc845c9644fad3bf0Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:58507) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 4. Add imdb features to movies collection

In [62]:
Mongo.command('movies', """
db.getCollection('movies').aggregate([
    {
        $lookup:
          {
            from: "links",
            foreignField: "movieId",
            localField: "movieId", 
            as: "links"
          }
     },
     { $match: { links: { $exists: true, $not: {$size: 0} } } },
     { 
        $project: { 
            id: "$movieId",
            tmdb_id:      { "$arrayElemAt": ["$links.tmdbId", 0] },
            imdb_id:      { "$arrayElemAt": ["$links.imdbId", 0] },
            title:        { $arrayElemAt:   [ {$split:["$title","("]} ,  0 ] },
            release_year: { $arrayElemAt:   [ {$split:["$title","("]} ,  1 ] },
            genres:       { $split:         [ "$genres", "|" ] }
        } 
    }, 
    {
        $lookup:
          {
            from: "movies_metadata_v2",
            foreignField: "imdb_id",
            localField: "imdb_id", 
            as: "movies_metadata"
          }
     },
     { $match: {  movies_metadata: { $exists: true, $not: {$size: 0} } } },
     { 
        $project: { 
            id: 1,
            tmdb_id: 1,
            imdb_id: 1,
            title: 1,
            genres: 1,
            for_adults:        { "$arrayElemAt": ["$movies_metadata.adult", 0] },
            budget:            { "$arrayElemAt": ["$movies_metadata.budget", 0] },
            original_language: { "$arrayElemAt": ["$movies_metadata.original_language", 0] },
            overview:          { "$arrayElemAt": ["$movies_metadata.overview", 0] },
            poster:            { "$arrayElemAt": ["$movies_metadata.poster_path", 0] },
            release:           { "$arrayElemAt": ["$movies_metadata.release_date", 0] },
            popularity:        { "$arrayElemAt": ["$movies_metadata.popularity", 0] },
            vote_mean:         { "$arrayElemAt": ["$movies_metadata.vote_average", 0] },
            vote_count:        { "$arrayElemAt": ["$movies_metadata.vote_count", 0] }
        }
    },
    { $out: "movies_v2" }
]);
""")

2023-11-05 21:31:43,615 - INFO - Success: b'(node:58547) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654833e62dfd22febd33027eConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:58547) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 5. Group tags per used, movie pair.

In [30]:
Mongo.command('movies', """
db.getCollection('tags').aggregate(
    [
        { 
            $group: {
                _id: {
                    user_id: "$userId",
                    movie_id: "$movieId"
                },
                tags: { $push: { $toLower: "$tag" } }
            }
        },
        {
          $project: {
            _id: 0,
            user_id: "$_id.user_id",
            movie_id: "$_id.movie_id",
            user_movie_id: { $concat: [ { $toString: "$_id.user_id" } , "_", { $toString:"$_id.movie_id"} ] },
            tags: 1
          }  
        },
        { $out: "tags_v2" }
    ]
);
""")

2023-11-05 14:28:16,786 - INFO - Success: b'(node:75471) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  6547d0a7f8b59757cdc3e84cConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:75471) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 6. Create used_movie_id into new tags_v2 collection.

In [31]:
Mongo.command('movies', """
db.getCollection('tags_v2').createIndex(
    { 'user_movie_id': 1 }, 
    { unique: true, name: 'id_unique_index' }
)
""")

2023-11-05 14:28:35,086 - INFO - Success: b'(node:75570) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  6547d0c2348199d331a0436aConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:75570) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 7. Add used_movie_id fields into new ratings_v2 collection and also create a unique index.

In [32]:
Mongo.command('movies', """
db.getCollection('ratings').aggregate([
        {
          $project: {
            user_id: "$userId",
            movie_id: "$movieId",
            user_movie_id: { $concat: [ { $toString: "$userId" } , "_", { $toString:"$movieId"} ] },
            rating: 1,
            timestamp: 1
          }  
        },
        { $out: "ratings_v2" }
    ]
);
""")

2023-11-05 14:30:41,930 - INFO - Success: b'(node:75704) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  6547d0e558a107a67f388f6fConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:75704) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [46]:
Mongo.command('movies', """
// Group doc ids with same user_movie_id.
duplicates = db.getCollection('ratings_v2').aggregate([
  {
    $group: {
      _id:   { field: "$user_movie_id" },
      count: { $sum: 1 },
      docs:  { $push: "$_id" }
    }
  },
  {
    $match: {
      count: { $gt: 1 }
    }
  }
]);


// Paso 2: Crea una instancia de Bulk y agrega las operaciones de eliminación
var bulk = db.getCollection('ratings_v2').initializeUnorderedBulkOp();

duplicates.forEach(function(duplicate) {
  var idsToDelete = duplicate.docs.slice(1); // conserva el primer documento y elimina los demás
  bulk.find({ _id: { $in: idsToDelete } }).remove();
});

// Paso 3: Ejecuta las operaciones de eliminación en lote
bulk.execute();
""")

2023-11-05 21:07:19,775 - INFO - Success: b'(node:49990) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  65482b5f41030d440b64aca6Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:49990) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [48]:
Mongo.command('movies', """
db.getCollection('ratings_v2').createIndex(
    { 'user_movie_id': 1 }, 
    { unique: true, name: 'id_unique_index' }
);
""")

2023-11-05 21:15:29,648 - INFO - Success: b'(node:54767) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  6548300ba1daa7a3dfd145cbConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:54767) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 8. Join ratting_v2 and tags_v2 collections by user_movie_id into a new ratings_tags_v1 collection.

In [49]:
Mongo.command('movies', """
db.getCollection('ratings_v2').aggregate([
    {
        $lookup:
          {
            from: "tags_v2",
            foreignField: "user_movie_id",
            localField: "user_movie_id", 
            as: "tags_v2"
          }
     },
     { $match: { tags_v2: { $exists: true, $not: {$size: 0} } } },
     { 
        $project: { 
            user_id: 1,
            movie_id: 1,
            rating: 1,
            timestamp: 1,
            tags: "$tags_v2.tags"
        }
    },
    {
        $addFields: {  
            _id: { $concat: [ { $toString: "$user_id" } , "_", { $toString:"$movie_id"} ] },            
            tags: {
                "$reduce": {
                    "input": "$tags",
                    "initialValue": [],
                    "in": { "$setUnion": [ "$$value", "$$this" ] }
                }
            }
        }
    },
    { $out: "ratings_tags_v1" }
]);
""")

2023-11-05 21:17:38,319 - INFO - Success: b'(node:55035) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  65483056fd265e06abb697f6Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:55035) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 9. Add tags field into movie_v2 collections:

In [51]:
Mongo.command('movies', """
db.getCollection('movies_v2').createIndex(
    { 'id': 1 }, 
    { unique: true, name: 'id_unique_index' }
)
""")

2023-11-05 21:23:18,985 - INFO - Success: b'(node:56675) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654831f6ba31b73c73aa0f02Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:56675) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [54]:
Mongo.command('movies', """

// Group doc ids with same user_movie_id.
duplicates = db.getCollection('tags_v2').aggregate([
  {
    $group: {
      _id:   { field: "$movie_id" },
      count: { $sum: 1 },
      docs:  { $push: "$_id" }
    }
  },
  {
    $match: {
      count: { $gt: 1 }
    }
  }
]);


// Paso 2: Crea una instancia de Bulk y agrega las operaciones de eliminación
var bulk = db.getCollection('tags_v2').initializeUnorderedBulkOp();

duplicates.forEach(function(duplicate) {
  var idsToDelete = duplicate.docs.slice(1); // conserva el primer documento y elimina los demás
  bulk.find({ _id: { $in: idsToDelete } }).remove();
});

// Paso 3: Ejecuta las operaciones de eliminación en lote
bulk.execute();
""")

2023-11-05 21:26:30,617 - INFO - Success: b'(node:57336) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654832b28d623889557fcaceConnecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:57336) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [55]:
Mongo.command('movies', """
db.getCollection('tags_v2').createIndex(
    { 'movie_id': 1 }, 
    { unique: true, name: 'id_unique_index' }
)
""")

2023-11-05 21:26:43,545 - INFO - Success: b'(node:57400) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654832c35f7a69d6718b0907Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:57400) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


In [56]:
Mongo.command('movies', """
db.getCollection('movies_v2').aggregate([
    {
        $lookup:
          {
            from: "tags_v2",
            foreignField: "movie_id",
            localField: "id",
            as: "tags_v2"
          }
    },
    { $match: { tags_v2: { $exists: true, $not: {$size: 0} } } },
    { 
        $addFields: { 
            tags: {
                "$reduce": {
                    "input": "$tags_v2.tags",
                    "initialValue": [],
                    "in": { "$setUnion": [ "$$value", "$$this" ] }
                }
            }
        }
    },
    { $unset: ["tags_v2"] },
    { $addFields: {  _id: "$id" } },
    { $unset: ["id"] },
    { $out: "movies_v3" }
]);
""")

2023-11-05 21:27:06,336 - INFO - Success: b'(node:57480) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.(Use `node --trace-deprecation ...` to show where the warning was created)Current Mongosh Log ID:  654832da3c35bced08000894Connecting to:    mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.0.2Using MongoDB:    7.0.2Using Mongosh:    2.0.2For mongosh info see: https://docs.mongodb.com/mongodb-shell/------   The server generated these startup warnings when booting   2023-11-05T11:01:27.653-03:00: Access control is not enabled for the database. Read and write access to data and configuration is unrestricted------Loading file: /tmp/query.js(node:57480) [DEP0174] DeprecationWarning: Calling promisify on a function that returns a Promise is likely a mistake.'


#### 10. Export final ccollections to json files:

* movies_v3 to movies_v3.json
* ratings_tags_v1 to ratings_tags_v1.json

In [ ]:
Mongo.export_to_json(database='movies', path=DATASETS_PATH, collections=['movies_v3'])

In [10]:
Mongo.export_to_json(database='movies', path=DATASETS_PATH, collections=['ratings_tags_v1'])

2022-04-16 11:21:33,739 - ERROR - b'2022-04-16T11:21:31.332-0300\tconnected to: mongodb://localhost/\n2022-04-16T11:21:32.333-0300\t[##########..............]  movies.ratings_tags_v1  88000/210725  (41.8%)\n2022-04-16T11:21:33.333-0300\t[####################....]  movies.ratings_tags_v1  176000/210725  (83.5%)\n2022-04-16T11:21:33.737-0300\t[########################]  movies.ratings_tags_v1  210725/210725  (100.0%)\n2022-04-16T11:21:33.737-0300\texported 210725 records\n'
